# Project proposal report - Snehal Thorat

### The following steps have been performed in Jupyter notebook running of EMR/EC2:
Spark session - user PySpark3 kernel - your Spark sessions is automaticly set to "spark"

Read json file and store into Spark Dataframe

Create parquet file

Create Spark Dataframe and load the s3 parquet file pointing to my s3 bucket created in initial Jupyter notebook

Create table poiniting to this Dataframe

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

"""
import pandas as pd
#from IPython.core.display import display, HTML
#display(HTML('<h1>Hello, world!</h1>'))
"""

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1554478263417_0001,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
"\nimport pandas as pd\n#from IPython.core.display import display, HTML\n#display(HTML('<h1>Hello, world!</h1>'))\n"

### Show current list of Hive tables

In [2]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

In [3]:
#spark.sql("drop database cust_complains")
spark.sql("create database cust_complains")
spark.sql("show databases").show()

+--------------+
|  databaseName|
+--------------+
|cust_complains|
|       default|
+--------------+

In [4]:
dfpq = spark.read.parquet("s3://snehalpreprocessedfiles/complaints030119.pq/part-00000-b286b6dc-956f-4ff7-8f4e-6ba555b60ed9-c000.snappy.parquet")

In [5]:
dfpq.write.saveAsTable("cust_complains.complaints")

## Create table in Hive and Save Data from Spark DataFRame into Hive

In [7]:
spark.sql("show tables in cust_complains").show()

+--------------+----------+-----------+
|      database| tableName|isTemporary|
+--------------+----------+-----------+
|cust_complains|complaints|      false|
+--------------+----------+-----------+

In [8]:
#Lets see Hive Table information COLUMN names
spark.sql("desc cust_complains.complaints").show(n=100,truncate= False)

+-------------------------+---------+-------+
|col_name                 |data_type|comment|
+-------------------------+---------+-------+
|company                  |string   |null   |
|company_public_response  |string   |null   |
|company_response         |string   |null   |
|complaint_id             |string   |null   |
|complaint_what_happened  |string   |null   |
|consumer_consent_provided|string   |null   |
|consumer_disputed        |string   |null   |
|date_received            |string   |null   |
|date_sent_to_company     |string   |null   |
|issue                    |string   |null   |
|product                  |string   |null   |
|state                    |string   |null   |
|sub_issue                |string   |null   |
|sub_product              |string   |null   |
|submitted_via            |string   |null   |
|timely                   |string   |null   |
|tags                     |string   |null   |
|zip_code                 |string   |null   |
+-------------------------+-------

### Date range: As per the project proposal document used date range from 3/1/2018 (including) to 3/1/2019 (including)

In [9]:
spark.sql("SELECT MIN(date_received), MAX(date_received), MIN(date_sent_to_company), MAX(date_sent_to_company) FROM cust_complains.complaints").show(truncate=False)

+-------------------------+-------------------------+-------------------------+-------------------------+
|min(date_received)       |max(date_received)       |min(date_sent_to_company)|max(date_sent_to_company)|
+-------------------------+-------------------------+-------------------------+-------------------------+
|2018-03-01T12:00:00-05:00|2019-03-01T12:00:00-05:00|2018-03-01T12:00:00-05:00|2019-03-28T12:00:00-05:00|
+-------------------------+-------------------------+-------------------------+-------------------------+

# Frequency of categorical data & top (20) most occurring values in every column and the contribution of each of those values to that column (in percentage)

In [9]:
spark.sql("SELECT product,COUNT(product) Number_of_complaints, COUNT(product)/250990*100 Percent_of_Total FROM cust_complains.complaints GROUP BY product ORDER BY Number_of_complaints DESC").show(truncate=False)

+----------------------------------------------------------------------------+--------------------+------------------+
|product                                                                     |Number_of_complaints|Percent_of_Total  |
+----------------------------------------------------------------------------+--------------------+------------------+
|Credit reporting, credit repair services, or other personal consumer reports|110644              |44.08303119646201 |
|Debt collection                                                             |48889               |19.478465277501094|
|Credit card or prepaid card                                                 |23881               |9.514721702059843 |
|Mortgage                                                                    |23783               |9.475676321765809 |
|Checking or savings account                                                 |20987               |8.361687716642097 |
|Student loan                                   

In [10]:
spark.sql("SELECT sub_product,COUNT(sub_product) Number_of_complaints, COUNT(sub_product)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY sub_product ORDER BY Number_of_complaints DESC").show(truncate=False)

+------------------------------------------+--------------------+------------------+
|sub_product                               |Number_of_complaints|Percent_of_Total  |
+------------------------------------------+--------------------+------------------+
|Credit reporting                          |108657              |43.291366189888045|
|General-purpose credit card or charge card|18648               |7.429778078807921 |
|Checking account                          |15637               |6.2301286903860715|
|Other debt                                |14475               |5.7671620383282205|
|Conventional home mortgage                |13606               |5.420933104904578 |
|Credit card debt                          |10754               |4.284632853898562 |
|I do not know                             |10293               |4.100960197617435 |
|Medical debt                              |7629                |3.0395633292163033|
|Federal student loan servicing            |5629                |

In [112]:
spark.sql("SELECT company,COUNT(company) Number_of_complaints, COUNT(company)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY company ORDER BY Number_of_complaints DESC").show(truncate=False)
spark.sql("SELECT company_public_response,COUNT(company_public_response) Number_of_complaints, COUNT(company_public_response)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY company_public_response ORDER BY Number_of_complaints DESC").show()

+--------------------------------------+--------------------+------------------+
|company                               |Number_of_complaints|Percent_of_Total  |
+--------------------------------------+--------------------+------------------+
|EQUIFAX, INC.                         |30295               |12.070202000079684|
|Experian Information Solutions Inc.   |29769               |11.860631897685167|
|TRANSUNION INTERMEDIATE HOLDINGS, INC.|28398               |11.314394995816567|
|JPMORGAN CHASE & CO.                  |8794                |3.503725248017849 |
|WELLS FARGO & COMPANY                 |8552                |3.4073070640264556|
|BANK OF AMERICA, NATIONAL ASSOCIATION |7727                |3.078608709510339 |
|CAPITAL ONE FINANCIAL CORPORATION     |7536                |3.0025100601617596|
|CITIBANK, N.A.                        |7111                |2.8331806048049724|
|Navient Solutions, LLC.               |5046                |2.010438662894936 |
|SYNCHRONY FINANCIAL        

In [113]:
spark.sql("SELECT company_response,COUNT(company_response) Number_of_complaints, COUNT(company_response)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY company_response ORDER BY Number_of_complaints DESC").show(truncate=False)
#spark.sql("SELECT complaint_what_happened,COUNT(complaint_what_happened) FROM cust_complains.complaints GROUP BY complaint_what_happened").show(truncate=False)

+-------------------------------+--------------------+------------------+
|company_response               |Number_of_complaints|Percent_of_Total  |
+-------------------------------+--------------------+------------------+
|Closed with explanation        |201381              |80.23467070401212 |
|Closed with non-monetary relief|33729               |13.438383999362525|
|Closed with monetary relief    |11577               |4.612534363918881 |
|In progress                    |2677                |1.0665763576238096|
|Untimely response              |1624                |0.647037730586876 |
|null                           |0                   |0.0               |
+-------------------------------+--------------------+------------------+

In [119]:
spark.sql("SELECT consumer_consent_provided,COUNT(consumer_consent_provided) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY consumer_consent_provided ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------------------+--------------------+-------------------+
|consumer_consent_provided|Number_of_complaints|Percent_of_Total   |
+-------------------------+--------------------+-------------------+
|Consent provided         |107419              |42.798119446989915 |
|Consent not provided     |78236               |31.17096298657317  |
|N/A                      |46610               |18.57046097454082  |
|Other                    |7057                |2.811665803418463  |
|Consent withdrawn        |269                 |0.10717558468464879|
|null                     |0                   |4.54161520379298   |
+-------------------------+--------------------+-------------------+

In [120]:
spark.sql("SELECT consumer_disputed,COUNT(consumer_disputed)  Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total FROM cust_complains.complaints GROUP BY consumer_disputed ORDER BY Number_of_complaints DESC").show(truncate=False)

+-----------------+--------------------+----------------+
|consumer_disputed|Number_of_complaints|Percent_of_Total|
+-----------------+--------------------+----------------+
|N/A              |250990              |100.0           |
+-----------------+--------------------+----------------+

In [121]:
spark.sql("SELECT date_received,COUNT(date_received) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY date_received ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------------------+--------------------+-------------------+
|date_received            |Number_of_complaints|Percent_of_Total   |
+-------------------------+--------------------+-------------------+
|2018-04-05T12:00:00-05:00|1260                |0.5020120323518865 |
|2018-04-10T12:00:00-05:00|1178                |0.4693414080242241 |
|2018-04-24T12:00:00-05:00|1141                |0.45459978485198616|
|2018-03-06T12:00:00-05:00|1103                |0.4394597394318499 |
|2018-04-11T12:00:00-05:00|1097                |0.4370692059444599 |
|2018-05-16T12:00:00-05:00|1069                |0.42591338300330694|
|2018-04-06T12:00:00-05:00|1063                |0.42352284951591695|
|2018-04-04T12:00:00-05:00|1057                |0.4211323160285271 |
|2018-03-22T12:00:00-05:00|1055                |0.4203354715327304 |
|2018-10-16T12:00:00-05:00|1038                |0.4135622933184589 |
|2018-03-19T12:00:00-05:00|1037                |0.4131638710705605 |
|2018-04-03T12:00:00-05:00|1026   

In [122]:
spark.sql("SELECT date_sent_to_company,COUNT(date_sent_to_company) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY date_sent_to_company ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------------------+--------------------+-------------------+
|date_sent_to_company     |Number_of_complaints|Percent_of_Total   |
+-------------------------+--------------------+-------------------+
|2018-04-10T12:00:00-05:00|1183                |0.4713335192637157 |
|2018-04-24T12:00:00-05:00|1128                |0.44942029562930796|
|2018-04-11T12:00:00-05:00|1128                |0.44942029562930796|
|2018-05-16T12:00:00-05:00|1119                |0.44583449539822306|
|2018-12-11T12:00:00-05:00|1088                |0.433483405713375  |
|2018-12-18T12:00:00-05:00|1066                |0.424718116259612  |
|2018-03-22T12:00:00-05:00|1059                |0.42192916052432367|
|2018-03-21T12:00:00-05:00|1044                |0.4159528268058489 |
|2019-01-30T12:00:00-05:00|1038                |0.4135622933184589 |
|2018-06-25T12:00:00-05:00|1033                |0.4115701820789673 |
|2018-12-12T12:00:00-05:00|1030                |0.4103749153352723 |
|2018-05-14T12:00:00-05:00|1028   

In [123]:
spark.sql("SELECT issue,COUNT(issue)  Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total FROM cust_complains.complaints GROUP BY issue ORDER BY Number_of_complaints DESC").show(truncate=False)

+--------------------------------------------------------------------------------+--------------------+------------------+
|issue                                                                           |Number_of_complaints|Percent_of_Total  |
+--------------------------------------------------------------------------------+--------------------+------------------+
|Incorrect information on your report                                            |70264               |27.994740826327742|
|Problem with a credit reporting company's investigation into an existing problem|25663               |10.224710147814655|
|Attempts to collect debt not owed                                               |21962               |8.750149408342962 |
|Managing an account                                                             |13248               |5.278297940156978 |
|Improper use of your report                                                     |12582               |5.012948723056695 |
|Written notific

In [124]:
spark.sql("SELECT sub_issue,COUNT(sub_issue) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY sub_issue ORDER BY Number_of_complaints DESC").show(truncate=False)

+--------------------------------------------------------------------------------+--------------------+------------------+
|sub_issue                                                                       |Number_of_complaints|Percent_of_Total  |
+--------------------------------------------------------------------------------+--------------------+------------------+
|Information belongs to someone else                                             |33233               |13.240766564404957|
|Their investigation did not fix an error on your report                         |18677               |7.4413323239969715|
|Account status incorrect                                                        |13822               |5.506992310450616 |
|Account information incorrect                                                   |12059               |4.804573887405873 |
|Debt is not yours                                                               |11566               |4.608151719192    |
|Credit inquirie

In [125]:
spark.sql("SELECT state,COUNT(state)  Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total FROM cust_complains.complaints GROUP BY state ORDER BY Number_of_complaints DESC").show(truncate=False)

+-----+--------------------+------------------+
|state|Number_of_complaints|Percent_of_Total  |
+-----+--------------------+------------------+
|CA   |31718               |12.637156858838997|
|FL   |26624               |10.607593928044942|
|TX   |23744               |9.460137854097773 |
|NY   |16896               |6.731742300490059 |
|GA   |15046               |5.994661141878162 |
|IL   |10685               |4.257141718793577 |
|NC   |8721                |3.4746404239212723|
|NJ   |8525                |3.3965496633332   |
|PA   |8262                |3.2917646121359416|
|OH   |7331                |2.9208334993426033|
|VA   |6116                |2.4367504681461414|
|MD   |5729                |2.2825610582094904|
|MI   |5249                |2.0913183792182957|
|AZ   |4906                |1.954659548189171 |
|SC   |4581                |1.8251723176222159|
|TN   |4361                |1.737519423084585 |
|MO   |3974                |1.5833300131479342|
|MA   |3790                |1.5100203195

In [126]:
spark.sql("SELECT submitted_via,COUNT(submitted_via) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY submitted_via ORDER BY Number_of_complaints DESC").show(truncate=False)

+-------------+--------------------+-------------------+
|submitted_via|Number_of_complaints|Percent_of_Total   |
+-------------+--------------------+-------------------+
|Web          |204380              |81.42953902545919  |
|Referral     |21614               |8.611498466074345  |
|Phone        |11954               |4.76273955137655   |
|Postal mail  |8272                |3.295748834614925  |
|Fax          |4751                |1.892904099764931  |
|Email        |19                  |0.00757002271006813|
+-------------+--------------------+-------------------+

In [127]:
spark.sql("SELECT timely,COUNT(timely) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY timely ORDER BY Number_of_complaints DESC").show(truncate=False)

+------+--------------------+------------------+
|timely|Number_of_complaints|Percent_of_Total  |
+------+--------------------+------------------+
|Yes   |246586              |98.24534842025578 |
|No    |4404                |1.7546515797442128|
+------+--------------------+------------------+

In [128]:
spark.sql("SELECT tags,COUNT(tags) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total  FROM cust_complains.complaints GROUP BY tags ORDER BY Number_of_complaints DESC").show(truncate=False)

+-----------------------------+--------------------+------------------+
|tags                         |Number_of_complaints|Percent_of_Total  |
+-----------------------------+--------------------+------------------+
|Servicemember                |22547               |8.98322642336348  |
|Older American               |7197                |2.867444918124228 |
|Older American, Servicemember|2578                |1.0271325550818757|
|null                         |0                   |87.12219610343041 |
+-----------------------------+--------------------+------------------+

In [108]:
spark.sql("SELECT zip_code,COUNT(zip_code) Number_of_complaints, COUNT(zip_code)/250990*100 Percent_of_Total FROM cust_complains.complaints GROUP BY zip_code ORDER BY Number_of_complaints DESC").show(truncate=False)

+--------+--------------------+-------------------+
|zip_code|Number_of_complaints|Percent_of_Total   |
+--------+--------------------+-------------------+
|300XX   |2115                |0.8426630543049525 |
|330XX   |1920                |0.7649707159647795 |
|770XX   |1860                |0.74106538109088   |
|331XX   |1650                |0.6573967090322324 |
|606XX   |1455                |0.5797043706920594 |
|750XX   |1331                |0.5303000119526674 |
|334XX   |1328                |0.5291047452089724 |
|303XX   |1286                |0.5123710107972429 |
|112XX   |1160                |0.4621698075620542 |
|333XX   |1140                |0.45420136260408783|
|900XX   |1106                |0.4406550061755448 |
|302XX   |1051                |0.4187417825411371 |
|891XX   |1025                |0.4083828040957807 |
|945XX   |997                 |0.3972269811546277 |
|282XX   |887                 |0.35340053388581216|
|191XX   |855                 |0.34065102195306585|
|301XX   |82

In [17]:
#spark.sql("SELECT complaint_id,COUNT(complaint_id) FROM cust_complains.complaints GROUP BY complaint_id").show(truncate=False)

In [104]:
#spark.sql("SELECT product,COUNT(product) num_product, sub_product, COUNT(sub_product) num_sub_product FROM cust_complains.complaints GROUP BY product, sub_product ORDER BY num_product DESC, num_sub_product DESC").show()

total = spark.sql("SELECT t2.total FROM (SELECT count(*) total FROM cust_complains.complaints)t2").show()

spark.sql("SELECT c2.sub_product, COUNT(c2.sub_product) Number_of_complaints, COUNT(c2.sub_product)/250990*100 Percent_of_Total FROM cust_complains.complaints c2 GROUP BY c2.sub_product ORDER BY Number_of_complaints DESC").show(50,truncate=False)


+------+
| total|
+------+
|250990|
+------+

+------------------------------------------+--------------------+---------------------+
|sub_product                               |Number_of_complaints|percentOfTotal       |
+------------------------------------------+--------------------+---------------------+
|Credit reporting                          |108657              |43.291366189888045   |
|General-purpose credit card or charge card|18648               |7.429778078807921    |
|Checking account                          |15637               |6.2301286903860715   |
|Other debt                                |14475               |5.7671620383282205   |
|Conventional home mortgage                |13606               |5.420933104904578    |
|Credit card debt                          |10754               |4.284632853898562    |
|I do not know                             |10293               |4.100960197617435    |
|Medical debt                              |7629                |3.0395633

# Cross-frequency of some of the fields (group by multiple fields)

In [137]:
spark.sql("SELECT t1.product, c1.sub_product, COUNT(c1.sub_product)  Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total FROM cust_complains.complaints c1 CROSS JOIN (SELECT DISTINCT c2.product FROM cust_complains.complaints c2)t1 ON t1.product = c1.product GROUP BY t1.product, c1.sub_product ORDER BY product, Number_of_Complaints DESC").show(55)

+--------------------+--------------------+--------------------+--------------------+
|             product|         sub_product|Number_of_complaints|    Percent_of_Total|
+--------------------+--------------------+--------------------+--------------------+
|Checking or savin...|    Checking account|               15637|  6.2301286903860715|
|Checking or savin...|Other banking pro...|                3325|  1.3247539742619228|
|Checking or savin...|     Savings account|                1399|  0.5573927248097534|
|Checking or savin...|CD (Certificate o...|                 609| 0.24263914897007852|
|Checking or savin...|Personal line of ...|                  17|0.006773178214271485|
|Credit card or pr...|General-purpose c...|               18648|   7.429778078807921|
|Credit card or pr...|   Store credit card|                3917|  1.5606199450177298|
|Credit card or pr...|General-purpose p...|                 802| 0.31953464281445476|
|Credit card or pr...|Government benefi...|           

In [139]:
spark.sql("SELECT t1.issue, c1.sub_issue, COUNT(c1.sub_issue) Number_of_complaints, COUNT(*)/250990*100 Percent_of_Total FROM cust_complains.complaints c1 CROSS JOIN (SELECT DISTINCT c2.issue FROM cust_complains.complaints c2)t1 ON t1.issue = c1.issue GROUP BY t1.issue, c1.sub_issue ORDER BY issue, Number_of_Complaints DESC").show()

+--------------------+--------------------+--------------------+--------------------+
|               issue|           sub_issue|Number_of_complaints|    Percent_of_Total|
+--------------------+--------------------+--------------------+--------------------+
|         Advertising|Confusing or misl...|                  31|0.012351089684848002|
|         Advertising|Changes in terms ...|                  10|0.003984222478983226|
|Advertising and m...|Didn't receive ad...|                 883| 0.35180684489421893|
|Advertising and m...|Confusing or misl...|                 549| 0.21873381409617915|
|Applying for a mo...|                null|                   0|  0.0792860273317662|
|Applying for a mo...|                null|                   0|  0.7378780031076935|
|Attempts to colle...|   Debt is not yours|               11566|      4.608151719192|
|Attempts to colle...|Debt was result o...|                5073|   2.021196063588191|
|Attempts to colle...|       Debt was paid|           

# Finding missing (NULL) values for each column

In [14]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE complaint_id IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [15]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE product IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [16]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE sub_product IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [17]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE company IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [18]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE company_public_response IS NULL").show()

+--------+
|count(1)|
+--------+
|  125218|
+--------+

In [19]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE company_response IS NULL").show()

+--------+
|count(1)|
+--------+
|       2|
+--------+

In [20]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE complaint_what_happened IS NULL").show()

+--------+
|count(1)|
+--------+
|  143591|
+--------+

In [21]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE consumer_consent_provided IS NULL").show()

+--------+
|count(1)|
+--------+
|   11399|
+--------+

In [22]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE consumer_disputed IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [23]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE date_received IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [24]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE date_sent_to_company IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [25]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE issue IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [26]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE sub_issue IS NULL").show()

+--------+
|count(1)|
+--------+
|   34489|
+--------+

In [27]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE state IS NULL").show()

+--------+
|count(1)|
+--------+
|    7411|
+--------+

In [28]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE submitted_via IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [29]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE timely IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

In [30]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE tags IS NULL").show()

+--------+
|count(1)|
+--------+
|  218668|
+--------+

In [31]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE zip_code IS NULL").show()

+--------+
|count(1)|
+--------+
|   33646|
+--------+

In [32]:
spark.sql("SELECT count(*) FROM cust_complains.complaints WHERE complaint_id IS NULL").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+

# Distinct values for each column. Showing only top 20

In [33]:
spark.sql("SELECT DISTINCT complaint_id FROM cust_complains.complaints").show()

+------------+
|complaint_id|
+------------+
|     3166501|
|     3164583|
|     3162168|
|     3161707|
|     3159784|
|     3157845|
|     3156339|
|     3151053|
|     3160135|
|     3148837|
|     3142790|
|     3140870|
|     3140817|
|     3136573|
|     3135981|
|     3136430|
|     3136039|
|     3134676|
|     2835893|
|     2834917|
+------------+
only showing top 20 rows

In [34]:
spark.sql("SELECT DISTINCT product FROM cust_complains.complaints").show()

+--------------------+
|             product|
+--------------------+
|     Debt collection|
|Money transfer, v...|
|Checking or savin...|
|            Mortgage|
|Credit card or pr...|
|Credit reporting,...|
|Vehicle loan or l...|
|        Student loan|
|Payday loan, titl...|
+--------------------+

In [35]:
spark.sql("SELECT DISTINCT sub_product FROM cust_complains.complaints").show()

+--------------------+
|         sub_product|
+--------------------+
|                Loan|
|     Savings account|
|Conventional home...|
|CD (Certificate o...|
|               Lease|
|         Money order|
|           Gift card|
|    Payday loan debt|
|    Checking account|
|    Virtual currency|
|Mobile or digital...|
|       I do not know|
|         Payday loan|
|          Title loan|
|Foreign currency ...|
|Other banking pro...|
|General-purpose c...|
|International mon...|
|Other personal co...|
|          Other debt|
+--------------------+
only showing top 20 rows

In [36]:
spark.sql("SELECT DISTINCT company FROM cust_complains.complaints").show()

+--------------------+
|             company|
+--------------------+
|Eagle Home Mortga...|
|       PlusFour, Inc|
|   CU Recovery, Inc.|
|FORD MOTOR CREDIT...|
|National Recoveri...|
|ClearOne Advantag...|
|Landmark Mortgage...|
|Worldwide Process...|
|Manhattan Beach V...|
|Delta Management ...|
|F&L Marketing Ent...|
|ACOPIA CAPITAL GROUP|
|Zealandia Holding...|
|Praxis Financial ...|
|           AFNI INC.|
|Medical Data Syst...|
|Southern Credit R...|
|NAM National Arbi...|
|Capital Markets C...|
|Law Office of Har...|
+--------------------+
only showing top 20 rows

In [37]:
spark.sql("SELECT DISTINCT company_public_response FROM cust_complains.complaints").show()

+-----------------------+
|company_public_response|
+-----------------------+
|   Company disputes ...|
|   Company has respo...|
|   Company believes ...|
|                   null|
|   Company believes ...|
|   Company believes ...|
|   Company believes ...|
|   Company believes ...|
|   Company can't ver...|
|   Company believes ...|
+-----------------------+

In [38]:
spark.sql("SELECT DISTINCT company_response FROM cust_complains.complaints").show()

+--------------------+
|    company_response|
+--------------------+
|                null|
|   Untimely response|
|Closed with non-m...|
|         In progress|
|Closed with monet...|
|Closed with expla...|
+--------------------+

In [39]:
spark.sql("SELECT DISTINCT consumer_consent_provided FROM cust_complains.complaints").show()

+-------------------------+
|consumer_consent_provided|
+-------------------------+
|                     null|
|        Consent withdrawn|
|                    Other|
|         Consent provided|
|     Consent not provided|
|                      N/A|
+-------------------------+

In [40]:
spark.sql("SELECT DISTINCT consumer_disputed FROM cust_complains.complaints").show()

+-----------------+
|consumer_disputed|
+-----------------+
|              N/A|
+-----------------+

In [41]:
spark.sql("SELECT DISTINCT date_received FROM cust_complains.complaints").show()

+--------------------+
|       date_received|
+--------------------+
|2019-01-03T12:00:...|
|2018-08-02T12:00:...|
|2019-02-11T12:00:...|
|2018-08-19T12:00:...|
|2018-06-01T12:00:...|
|2018-08-29T12:00:...|
|2018-06-23T12:00:...|
|2018-03-20T12:00:...|
|2018-04-14T12:00:...|
|2018-05-22T12:00:...|
|2018-06-20T12:00:...|
|2018-12-24T12:00:...|
|2018-03-27T12:00:...|
|2019-01-31T12:00:...|
|2018-12-29T12:00:...|
|2019-01-29T12:00:...|
|2018-07-07T12:00:...|
|2018-07-01T12:00:...|
|2018-12-02T12:00:...|
|2018-11-29T12:00:...|
+--------------------+
only showing top 20 rows

In [42]:
spark.sql("SELECT DISTINCT date_sent_to_company FROM cust_complains.complaints").show()

+--------------------+
|date_sent_to_company|
+--------------------+
|2019-01-03T12:00:...|
|2018-08-02T12:00:...|
|2019-02-11T12:00:...|
|2018-06-01T12:00:...|
|2018-08-29T12:00:...|
|2018-08-19T12:00:...|
|2018-03-20T12:00:...|
|2018-06-23T12:00:...|
|2018-04-14T12:00:...|
|2018-05-22T12:00:...|
|2018-06-20T12:00:...|
|2018-12-24T12:00:...|
|2018-03-27T12:00:...|
|2019-03-05T12:00:...|
|2019-01-31T12:00:...|
|2018-12-29T12:00:...|
|2019-01-29T12:00:...|
|2018-07-07T12:00:...|
|2018-07-01T12:00:...|
|2018-12-02T12:00:...|
+--------------------+
only showing top 20 rows

In [43]:
spark.sql("SELECT DISTINCT issue FROM cust_complains.complaints").show()

+--------------------+
|               issue|
+--------------------+
|Communication tac...|
|Unexpected or oth...|
|Getting a loan or...|
|Vehicle was repos...|
|Confusing or miss...|
|   Property was sold|
|      Getting a loan|
|Other service pro...|
|Can't stop withdr...|
|Problem with a co...|
|Loan payment wasn...|
|Problem caused by...|
|Improper use of y...|
|Problem with a le...|
|Managing, opening...|
|Other features, t...|
|  Opening an account|
|Getting a credit ...|
|Problem with the ...|
|Charged fees or i...|
+--------------------+
only showing top 20 rows

In [44]:
spark.sql("SELECT DISTINCT sub_issue FROM cust_complains.complaints").show()

+--------------------+
|           sub_issue|
+--------------------+
|      Banking errors|
|      Privacy issues|
|Transaction was n...|
|Problem getting y...|
|Problem when atte...|
|Old information r...|
|Account opened as...|
|  Late or other fees|
|Account status in...|
|Deposits or withd...|
|Information is in...|
|Talked to a third...|
|Told you not to r...|
|Confusing or miss...|
|       Credit denial|
|         Denied loan|
|Credit card compa...|
|Company closed yo...|
|Impersonated atto...|
|Problem with rewa...|
+--------------------+
only showing top 20 rows

In [45]:
spark.sql("SELECT DISTINCT state FROM cust_complains.complaints").show()

+--------------------+
|               state|
+--------------------+
|UNITED STATES MIN...|
|                  AZ|
|                  SC|
|                  LA|
|                  MN|
|                  AA|
|                  NJ|
|                  DC|
|                  OR|
|                  VA|
|                null|
|                  RI|
|                  KY|
|                  WY|
|                  NH|
|                  MI|
|                  NV|
|                  WI|
|                  ID|
|                  CA|
+--------------------+
only showing top 20 rows

In [46]:
spark.sql("SELECT DISTINCT submitted_via FROM cust_complains.complaints").show()

+-------------+
|submitted_via|
+-------------+
|        Phone|
|          Fax|
|        Email|
|     Referral|
|  Postal mail|
|          Web|
+-------------+

In [47]:
spark.sql("SELECT DISTINCT timely FROM cust_complains.complaints").show()

+------+
|timely|
+------+
|    No|
|   Yes|
+------+

In [48]:
spark.sql("SELECT DISTINCT tags FROM cust_complains.complaints").show()

+--------------------+
|                tags|
+--------------------+
|                null|
|      Older American|
|       Servicemember|
|Older American, S...|
+--------------------+

In [49]:
spark.sql("SELECT DISTINCT zip_code FROM cust_complains.complaints").show()

+--------+
|zip_code|
+--------+
|   75007|
|   921XX|
|   068XX|
|   19132|
|   687XX|
|   89106|
|   36526|
|   92027|
|   78382|
|   60053|
|   11722|
|   803XX|
|   28117|
|   178XX|
|   66762|
|   35670|
|   33174|
|   61354|
|   90022|
|   28334|
+--------+
only showing top 20 rows

In [50]:
spark.sql("SELECT DISTINCT complaint_id FROM cust_complains.complaints").show()

+------------+
|complaint_id|
+------------+
|     3166501|
|     3164583|
|     3162168|
|     3161707|
|     3159784|
|     3157845|
|     3156339|
|     3151053|
|     3160135|
|     3148837|
|     3142790|
|     3140870|
|     3140817|
|     3136573|
|     3135981|
|     3136430|
|     3136039|
|     3134676|
|     2835893|
|     2834917|
+------------+
only showing top 20 rows